BIBLIOTECAS

In [56]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt
import numpy as np
import unidecode
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output
import dash_bootstrap_components as dbc

EXTRAÇÃO DOS DADOS

In [57]:
#   Dados de acesso PostgreSQL
trt_name = 'automatiza_metricas_dev'
trt_user = 'automatiza'
trt_password = 'automatiza'
trt_host = '10.5.15.78'
trt_port = '5432'

#   Acessa o banco de dados e cria a engine
db_url = f'postgresql+psycopg2://{trt_user}:{trt_password}@{trt_host}:{trt_port}/{trt_name}'
engine = create_engine(db_url)

#   Seleciona as tabelas
query_cartoes = "SELECT * FROM cartoes"
query_checklists = "SELECT * FROM checklists"
query_listas = "SELECT * FROM listas"
query_quadros = "SELECT * FROM quadros"

#   Armazena tabelas em dataframes
df_cartoes = pd.read_sql(query_cartoes, engine)
df_checklists = pd.read_sql(query_checklists, engine)
df_listas = pd.read_sql(query_listas, engine)
df_quadros = pd.read_sql(query_quadros, engine)

#   Copia de segurança das tabelas
quadros = df_quadros.copy()
listas = df_listas.copy()
cartoes = df_cartoes.copy()
checklists = df_checklists.copy()

TRANSFORMAÇÃO DOS DADOS

In [58]:
#   Renomear coluna "nome" das tabelas
quadros.rename(columns={'nome': 'quadro'}, inplace=True)
listas.rename(columns={'nome': 'lista'}, inplace=True)
cartoes.rename(columns={'nome': 'cartao'}, inplace=True)
checklists.rename(columns={'nome': 'checklist'}, inplace=True)

#   Concatenar dataframes
consolidado = quadros.merge(listas, on='id_quadro', how='outer') \
    .merge(cartoes, on='id_lista', how='outer') \
        .merge(checklists, on='id_cartao', how='outer')

#   Tratar valores nulos
consolidado['data_inicio'] = consolidado['data_inicio'].fillna(consolidado['data_fim'])
consolidado['data_fim'] = consolidado['data_fim'].fillna(consolidado['data_inicio'])
consolidado[['data_inicio', 'data_fim']] = consolidado[['data_inicio', 'data_fim']].fillna(pd.Timestamp('01-01-1900'))
consolidado['lead_time'] = consolidado['lead_time'].fillna(1)
consolidado['descricao'] = consolidado['descricao'].replace('', np.nan).fillna('sem descricao')
consolidado['id_lista'] = consolidado['id_lista'].replace('', np.nan).fillna('sem lista')
consolidado['lista'] = consolidado['lista'].replace('', np.nan).fillna('sem lista')
consolidado['id_cartao'] = consolidado['id_cartao'].replace('', np.nan).fillna('sem cartao')
consolidado['cartao'] = consolidado['cartao'].replace('', np.nan).fillna('sem cartao')
consolidado['id_checklist'] = consolidado['id_checklist'].replace('', np.nan).fillna('sem checklist')
consolidado['checklist'] = consolidado['checklist'].replace('', np.nan).fillna('sem checklist')
consolidado['quant_itens'] = consolidado['quant_itens'].replace('', np.nan).fillna(0)

#   Padronizar textos em series
def processar_texto(texto):
    texto = texto.lower() # letras minusculas
    texto = unidecode.unidecode(texto) # remove acentuação
    return texto
consolidado['lista'] = consolidado['lista'].apply(processar_texto)
consolidado['checklist'] = consolidado['checklist'].apply(processar_texto)

#   Converter tipos de dados
consolidado['data_inicio'] = pd.to_datetime(consolidado['data_inicio'], format='%d-%m-%Y')
consolidado['data_fim'] = pd.to_datetime(consolidado['data_fim'], format='%d-%m-%Y')
consolidado['lead_time'] = consolidado['lead_time'].astype(int)
consolidado['quant_itens'] = consolidado['quant_itens'].astype(int)

#   Tratar itens de checklist: bugs e testes
consolidado['checklist'] = consolidado['checklist'] \
    .replace(to_replace=r'.*bug.*', value='bug', regex=True) \
        .replace(to_replace=r'.*teste.*', value='teste', regex=True)



ANÁLISE EXPLORATÓRIA

In [59]:
#consolidado.to_csv('consolidado.csv', index=False)
#consolidado.to_excel('consolidado.xlsx', index=False)

#display(consolidado.info())
#display(consolidado.head(3))
#display(consolidado[consolidado['data_inicio'] == '01-01-1900'])
#display(consolidado['lista'].value_counts())
#display(consolidado[consolidado['quadro'] == 'AutomatizaTRT'].head(5))
#display(consolidado[consolidado['checklist'].str.contains('teste', case=False)])
#display(consolidado['quadro'][consolidado['checklist'].str.contains('teste', case=False)])
#display(consolidado['lead_time'].describe())
#display(consolidado['checklist'].value_counts())


CONFIGURAÇÕES DAS FIGURAS

In [60]:
#   Dados a serem inseridos nas figuras
tarefas_quadro = consolidado.groupby('quadro').size().reset_index(name = 'total_cartoes')
leadtime_tarefas = consolidado[['quadro', 'lead_time']]
checklist_interessada = consolidado[consolidado['checklist'].isin(['bug', 'teste'])]
bugs_encontrados = checklist_interessada['checklist'].str.contains('bug').sum()
testes_encontrados = checklist_interessada['checklist'].str.contains('teste').sum()

#   FIGURAS
#   Tarefas concluídas por quadro
fig_tarefasconcluidas = go.Figure(go.Bar(
    x = tarefas_quadro['total_cartoes'],
    y = tarefas_quadro['quadro'],
    orientation = 'h',
    text = tarefas_quadro['total_cartoes'],
    textposition = 'auto',
    textangle = 0
    ))
fig_tarefasconcluidas.update_layout(
    title = "TOTAL DE CARTÕES POR QUADRO",
    xaxis_title =  "TAREFAS CONCLUÍDAS",
    yaxis_title = "QUADROS",
    yaxis = dict(categoryorder = 'total ascending'),
    uniformtext_minsize = 8,
    uniformtext_mode = 'hide'
)

#   Tempo de tarefas concluídas
fig_leadtime = go.Figure(go.Scatter(
    y = leadtime_tarefas['lead_time'],
    mode = 'markers'
    ))
fig_leadtime.update_layout(
    title = "LEAD TIME",
    yaxis_title = "TEMPO DE TAREFA (DIAS)",
    xaxis_title = "TAREFAS CONCLUÍDAS"
)

#   Bugs e testes encontrados
fig_bugteste = go.Figure(go.Bar(
    x = checklist_interessada['checklist'],
    y = [bugs_encontrados, testes_encontrados],
    text = [bugs_encontrados, testes_encontrados]
    ))
fig_bugteste.update_layout(
    title = "BUGS E TESTES",
    xaxis_title =  "CHECKLISTS"
    
)


LAYOUT DA DASHBOARD

In [61]:
BS = "https://cdn.jsdelivr.net/npm/bootstrap@5.3.3/dist/css/bootstrap.min.css"
app = Dash(__name__, external_stylesheets=[BS])
app.layout = dbc.Container(
    [
        dbc.Row(
            [
                dbc.Col(
                    dcc.Graph(figure = fig_tarefasconcluidas), width={"size": 4}
                ),
                dbc.Col(
                    dcc.Graph(figure = fig_leadtime), width={"size": 4}
                ),
                dbc.Col(
                    dcc.Graph(figure = fig_bugteste), width={"size": 4}
                )
            ], justify='center'
        )
    ], fluid=True
)

if __name__ == '__main__':
    app.run_server(host='127.0.0.1', port=8050)